In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import stanza
import itertools
import requests

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import nltk

#stanza.download("uk")
#stanza.download("en")
#nltk.download('wordnet')

from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from nltk.corpus import wordnet 

def get_all_edges(concept):
    offset = 0
    req = requests.get('http://api.conceptnet.io/c/en/' + concept + '?offset=' + str(offset) + '&limit=1000').json()
    all_edges = req
    while len(req['edges']) == 1000:
        offset += 1000
        req = requests.get('http://api.conceptnet.io/c/en/' + concept + '?offset=' + str(offset) + '&limit=1000').json()
        all_edges['edges'] += req['edges']
    return all_edges

[nltk_data] Downloading package wordnet to /home/nata/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
my_stop_words = ["the", "a", "an", '.',',','!']

### Get data from https://nlp.stanford.edu/projects/snli/

In [3]:
train_path = "snli_1.0/snli_1.0/snli_1.0_train.jsonl"
test_path = "snli_1.0/snli_1.0/snli_1.0_test.jsonl"
dev_path = "snli_1.0/snli_1.0/snli_1.0_dev.jsonl"

In [4]:
def parse_data(file):    
    dict_list = []
    with open(file, "r") as read_file:
        for line in read_file.readlines():
            data = json.loads(line)
            if data["gold_label"]!='-':
                dict_list.append(data)
    return dict_list

### Create features

#### My progress:

**I iteration** : додала перші ж ознаки, які прийшли на думку, щоб мати певний мінімальний бенчмарк
1. кількість спільних слів у першому та другому реченні (n_common)
2. кількість слів у першому реченні (n_s1)
3. кількість слів у другому реченні (n_s2)

Я одразу ж отримала 0.47 accuracy на *dev* даних, що доволі непогано, порівнюючи з рандомним класифікатором, що дає 0.33 accuracy

**II iteration** : ознаки лексичної схожості. Спочатку пробувала використати лематизацію *stanza*. Коли я пробувала обробити кожне речення окремо, то він обробляв *test* вибірку більше 10 хв і не завершив обробку. Коли я пробувала обробити всі речення одночасно, то він ви'їв 10гб оперативки і аналогічно не закінчив обробку. Тому я вирішила перейти на щось легше і зупинилась на SnowballStemmer. Нові ознаки:
1. кількість спільних stemmed-слів(bigrams,trigrams) у першому та другому реченні (n_common_...)
2. кількість stemmed-слів(bigrams,trigrams) у першому реченні (n_1_...)
3. кількість stemmed-слів(bigrams,trigrams) у другому реченні (n_2_...)
4. частка спільних унікальних stemmed-слів(bigrams,trigrams) у першому та другому реченні з-поміж всіх унікальних stemmed-слів(bigrams,trigrams) (per_common_...)
5. частка stemmed-слів (bigrams,trigrams) у першому реченні з сумарних stemmed-слів (bigrams,trigrams) (per_1_...)
6. частка унікальних stemmed-слів (bigrams,trigrams) у першому реченні з сумарних унікальних stemmed-слів (bigrams,trigrams) (per_set_1...)
7. чи є слово "not" у першому реченні
8. чи є слово "not" у друг реченні

Я отримала 0.49 accuracy на *dev* даних

**III iteration** : ознаки граматичної схожості. Оскільки в даних вже є parse tree з розписаними частинами мови, я використала його. Нові ознаки:
1. Частка певної частини мови у кожному реченні: іменник, дієслово, прийменник, займенник (per_...)
2. Частка одинакових слів у двох реченнях з-поміж певної частини мови: іменник, дієслово, прийменник, чисельник (per_common_...)

Отримала 0.52 accuracy на *dev* даних.

**IV iteration** : ознаки семантичної схожості. Я хотіла в даному випадку використати ConceptNet і знайти всі можливі зв'язки і порахувати їх. Проте використати ConceptNet можна лише через запити, тому це займає дуже довго. В результаті я обмежилась лише пошуком синонімів (інших зв'язків там немає) за допомогою WordNet. Нижче є приклад використання ConceptNet і WordNet і час відпрацювання. звісно, щоб досягнути швидкості обробки, я втратила можливіть аналізувати більше зв'язків і точність впала. Нові ознаки:
1. Кількість синонімів у між двома реченнями (n_syn)

Отримала 0.52 accuracy на *dev* даних. (точність не змінилась)

Фінальні результати див нижче.

In [217]:
#nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma', tokenize_no_ssplit=True)
englishStemmer = SnowballStemmer("english")

def lex_features(s1,s2,name):
    f_names = {}
    f_names["n_common_"+name] = len(set(s1).intersection(set(s2)))
    f_names["n_1_"+name] = len(s1)
    f_names["n_2_"+name] = len(s2)
    if len(set(s1+s2))!=0:
        f_names["per_common_"+name] = len(set(s1).intersection(set(s2)))/len(set(s1+s2))
        f_names["per_1_"+name] = len(s1)/len(s1+s2)
        f_names["per_set_1_"+name] = len(set(s1))/len(set(s1+s2))
        #f_names["per_2_"+name] = len(s2)/len(s1+s2)   = 1 - per_1_
        #f_names["per_set_2_"+name] = len(set(s2))/len(set(s1+s2))    = 1 - per_set_1_
    else:
        f_names["per_common_"+name] = 0
        f_names["per_1_"+name] = 0
        f_names["per_set_1_"+name] = 0
    return f_names

def poc_intersection(s1,s2,POS):
    pos1 = re.findall(f"(\({POS}[^\(^\)]*\))", s1)
    pos2 = re.findall(f"(\({POS}[^\(^\)]*\))", s2)
    pos_words1 = [pair.split(" ")[1][:-1] for pair in pos1]
    pos_words2 = [pair.split(" ")[1][:-1] for pair in pos2]
    pos_words_stemmed1 = [englishStemmer.stem(word.lower()) for word in pos_words1]
    pos_words_stemmed2 = [englishStemmer.stem(word.lower()) for word in pos_words2]
    if len(set(pos_words_stemmed1+pos_words_stemmed2)) == 0:
        return 0
    
    return len(set(pos_words_stemmed1).intersection(set(pos_words_stemmed2)))/len(set(pos_words_stemmed1+pos_words_stemmed2))
    
def count_relations(s1,s2):
    relations = []
    for pair in list(itertools.product(s1, s2)):
        if pair[0]==pair[1]:
            continue
        all_concepts = get_all_edges(pair[0])
        for concept in all_concepts["edges"]:
            if concept["end"]["label"] == pair[1]:
                relations.append(concept["rel"]["label"])
    r,c = np.unique(relations, return_counts = True)
    relation_count_dict = {}
    for i in range(len(r)):
        relation_count_dict[r[i]] = c[i]
    return relation_count_dict

def count_synonyms(s1,s2):
    c = 0
    for word in s1:
        syns = wordnet.synsets(word) 
        synonims = set([s.lemmas()[0].name() for s in syns])
        synonims = synonims - set([word])
        if len(synonims.intersection(set(s2)))>0:
            c = c + 1
    return c

def get_features(d):
    s_token1 = [t.lower() for t in d["sentence1"].split(" ")]
    s_token2 = [t.lower() for t in d["sentence2"].split(" ")]
    s1 = list(set(s_token1) - set(my_stop_words))
    s2 = list(set(s_token2) - set(my_stop_words))
    
    stems1 = [englishStemmer.stem(token) for token in s1]
    stems2 = [englishStemmer.stem(token) for token in s2]
    
    two_grams1 = [" ".join(g) for g in ngrams(stems1,2)]
    two_grams2 = [" ".join(g) for g in ngrams(stems2,2)]
    
    three_grams1 = [" ".join(g) for g in ngrams(stems1,3)]
    three_grams2 = [" ".join(g) for g in ngrams(stems2,3)]
    
    f_names = {}
    
    # базові ознаки 
    f_names["n_common"] = len(set(s1).intersection(set(s2)))
    f_names["n_s1"] = len(s1)
    f_names["n_s2"] = len(s2)
    
    # ознаки лексичної схожості
    l_f = lex_features(stems1,stems2,"stem")
    l_f2 = lex_features(two_grams1,two_grams2,"two_grams")
    l_f3 = lex_features(three_grams1,three_grams2,"three_grams")
    f_names = {**f_names, **l_f, **l_f2, **l_f3}
    
    f_names["is_not_stems1"] = "not" in stems1
    f_names["is_not_stems2"] = "not" in stems2
    
    # ознаки граматичної схожості
    f_names["per_NN1"] = (d["sentence1_parse"].count("NN ")+d["sentence1_parse"].count("NNP ")+d["sentence1_parse"].count("NNS "))/len(s_token1)
    f_names["per_NN2"] = (d["sentence2_parse"].count("NN ")+d["sentence2_parse"].count("NNP ")+d["sentence2_parse"].count("NNS "))/len(s_token2)
    f_names["per_VB1"] = (d["sentence1_parse"].count("VB"))/len(s_token1)
    f_names["per_VB2"] = (d["sentence2_parse"].count("VB"))/len(s_token2)
    f_names["per_JJ1"] = (d["sentence1_parse"].count("JJ"))/len(s_token1)
    f_names["per_JJ2"] = (d["sentence2_parse"].count("JJ"))/len(s_token2)
    f_names["per_PRP1"] = (d["sentence1_parse"].count("PRP"))/len(s_token1)
    f_names["per_PRP2"] = (d["sentence2_parse"].count("PRP"))/len(s_token2)
    f_names["per_common_NN"] = poc_intersection(d["sentence1_parse"],d["sentence2_parse"],"NN")
    f_names["per_common_VB"] = poc_intersection(d["sentence1_parse"],d["sentence2_parse"],"VB")
    f_names["per_common_CD"] = poc_intersection(d["sentence1_parse"],d["sentence2_parse"],"CD")
    f_names["per_common_JJ"] = poc_intersection(d["sentence1_parse"],d["sentence2_parse"],"JJ")
    
    # ознаки семантичної схожості
    f_names["n_syn"] = count_synonyms(s1,s2)
    
    return f_names.values(), f_names.keys()

In [155]:
def get_X_y(data):
    X = []
    y = []
    for d in data:
        f, f_names = get_features(d)
        X.append(f)
        y.append(d["gold_label"])
    return pd.DataFrame(X, columns = f_names), np.array(y)

In [213]:
%%time
### Example of using ConceptNet and time 
c = count_relations(['Two','women','are','embracing','while','holding','to','go','packages'],
                    ['The','sisters','are','hugging','goodbye','while','holding','to','go','packages'])
c

CPU times: user 2.42 s, sys: 155 ms, total: 2.57 s
Wall time: 54.8 s


{'Synonym': 2}

In [214]:
%%time
### Example of using WordNet and time 
count_synonyms(['Two','women','are','embracing','while','holding','to','go','packages'],
              ['The','sisters','are','hugging','goodbye','while','holding','to','go','packages'])

CPU times: user 11.4 ms, sys: 3.88 ms, total: 15.3 ms
Wall time: 14.8 ms


0

In [218]:
%%time
data_test = parse_data(test_path)
X_test, y_test = get_X_y(data_test)
print("TEST shape:", X_test.shape, y_test.shape)

data_train = parse_data(train_path)
X_train, y_train = get_X_y(data_train)
print("TRAIN shape:", X_train.shape, y_train.shape)

data_dev = parse_data(dev_path)
X_dev, y_dev = get_X_y(data_dev)
print("TRAIN shape:", X_dev.shape, y_dev.shape)

TEST shape: (9824, 36) (9824,)
TRAIN shape: (549367, 36) (549367,)
TRAIN shape: (9842, 36) (9842,)
CPU times: user 6min 22s, sys: 3.7 s, total: 6min 26s
Wall time: 6min 27s


In [219]:
print(f"Total {len(X_train.columns)} features.")
X_train.columns

Total 36 features.


Index(['n_common', 'n_s1', 'n_s2', 'n_common_stem', 'n_1_stem', 'n_2_stem',
       'per_common_stem', 'per_1_stem', 'per_set_1_stem', 'n_common_two_grams',
       'n_1_two_grams', 'n_2_two_grams', 'per_common_two_grams',
       'per_1_two_grams', 'per_set_1_two_grams', 'n_common_three_grams',
       'n_1_three_grams', 'n_2_three_grams', 'per_common_three_grams',
       'per_1_three_grams', 'per_set_1_three_grams', 'is_not_stems1',
       'is_not_stems2', 'per_NN1', 'per_NN2', 'per_VB1', 'per_VB2', 'per_JJ1',
       'per_JJ2', 'per_PRP1', 'per_PRP2', 'per_common_NN', 'per_common_VB',
       'per_common_CD', 'per_common_JJ', 'n_syn'],
      dtype='object')

### Random classifier

In [220]:
dummy = DummyClassifier()
dummy.fit(X_train,y_train)
dev_preds_dummy = dummy.predict(X_dev)
print(classification_report(y_dev, dev_preds_dummy))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


               precision    recall  f1-score   support

contradiction       0.33      0.33      0.33      3278
   entailment       0.35      0.35      0.35      3329
      neutral       0.32      0.33      0.32      3235

     accuracy                           0.34      9842
    macro avg       0.34      0.34      0.34      9842
 weighted avg       0.34      0.34      0.34      9842



### Logistic regression

In [221]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train,y_train)
dev_preds = pipe.predict(X_dev)
train_preds = pipe.predict(X_train)
print("Metrics for DEV")
print(classification_report(y_dev, dev_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Metrics for DEV
               precision    recall  f1-score   support

contradiction       0.48      0.51      0.50      3278
   entailment       0.57      0.62      0.60      3329
      neutral       0.52      0.44      0.47      3235

     accuracy                           0.52      9842
    macro avg       0.52      0.52      0.52      9842
 weighted avg       0.52      0.52      0.52      9842



In [222]:
print("Metrics for TRAIN")
print(classification_report(y_train, train_preds))

Metrics for TRAIN
               precision    recall  f1-score   support

contradiction       0.48      0.52      0.50    183187
   entailment       0.55      0.61      0.58    183416
      neutral       0.52      0.43      0.47    182764

     accuracy                           0.52    549367
    macro avg       0.52      0.52      0.52    549367
 weighted avg       0.52      0.52      0.52    549367



In [223]:
### Feature importance
sorted(list(zip(np.absolute(pipe.named_steps["logisticregression"].coef_).sum(axis = 0), X_dev.columns)))

[(0.005658797375599877, 'is_not_stems1'),
 (0.038050000149370745, 'per_PRP1'),
 (0.059917332447525, 'per_VB1'),
 (0.060038447359466035, 'n_1_stem'),
 (0.060038447359466035, 'n_1_two_grams'),
 (0.060038447359466035, 'n_s1'),
 (0.06087236740131889, 'per_JJ1'),
 (0.0634655906730535, 'per_common_CD'),
 (0.0649716111478523, 'n_1_three_grams'),
 (0.09073235871119063, 'n_syn'),
 (0.0965806144060059, 'per_VB2'),
 (0.1202532584956459, 'is_not_stems2'),
 (0.14764025798170324, 'n_common_two_grams'),
 (0.1536043883507013, 'n_2_three_grams'),
 (0.18881596170279294, 'per_NN1'),
 (0.22998978708004114, 'n_2_stem'),
 (0.22998978708004114, 'n_s2'),
 (0.23297933770477736, 'n_2_two_grams'),
 (0.23974664493350847, 'per_PRP2'),
 (0.25194613424760004, 'per_NN2'),
 (0.2538109694704606, 'n_common_three_grams'),
 (0.2646084799407681, 'per_common_JJ'),
 (0.27344682934027537, 'per_common_three_grams'),
 (0.33703664049393073, 'per_common_VB'),
 (0.34066395040502484, 'per_JJ2'),
 (0.7012419183538059, 'per_1_three_g

### Model on test data

In [224]:
test_preds = pipe.predict(X_test)
print("Metrics for TEST")
print(classification_report(y_test, test_preds))

Metrics for TEST
               precision    recall  f1-score   support

contradiction       0.49      0.52      0.50      3237
   entailment       0.58      0.61      0.60      3368
      neutral       0.51      0.45      0.48      3219

     accuracy                           0.53      9824
    macro avg       0.53      0.53      0.53      9824
 weighted avg       0.53      0.53      0.53      9824



Фінальна точність на *test* даних рівна 0.53.

**Висновок:**
Загалом я змогла покращити точність за допомогою лінгвіністичних ознак схожості, проте є ще варіанти до покращення:

1. використати більше ознак семантичної схожості. Наприклад, AllenNLP точно має PythonAPI, проте не експериментувала, тому не знаю наскільки він швидкий
2. використати Text Similarity, наприклад відстань Левенштейна 
3. використати Word2Vec представлення даних (ае це не в рамках цього модуля)